In [16]:
import numpy as np
import scipy as sp
from scipy.stats import multivariate_normal
from scipy       import optimize
import scipy
import skimage
import matplotlib.pyplot as plt
import imageio
import pylab
from glob import glob
import sys
import itk
from itkwidgets import view
import math

from USresamplingutils import get_ruler_points_C52
from USresamplingutils import get_depth_and_zoom_C52

In [2]:
num_mask = 3
im_mask = []
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth5_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth12_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth16_Edited.png", itk.F)))
im_mask_depth = [5,12,16]

In [3]:
def get_top_and_bottom_curve_points_C52(im, centerX):
    """ Find points along the top (yt) and bottom (yb) curves of the ultrasound image """
    min_x = int(centerX-centerX*0.1)
    max_x = int(centerX+centerX*0.1)
    step_x = 10
    size_x = int((max_x - min_x)/step_x)
    xt = np.zeros(size_x, dtype=int)
    yt = np.zeros(size_x, dtype=int)
    for t in range(size_x):
        xt[t] = int(min_x + t * step_x)
        mid = np.mean(im[:,xt[t]-2:xt[t]+2],axis=1)
        nz = np.flatnonzero(mid)
        yt[t] = nz[0]
    min_x = int(centerX-centerX*0.25)
    max_x = int(centerX+centerX*0.25)
    step_x = 10
    size_x = int((max_x - min_x)/step_x)
    xb = np.zeros(size_x, dtype=int)
    yb = np.zeros(size_x, dtype=int)
    for t in range(size_x):
        xb[t] = int(min_x + t * step_x)
        mid = np.mean(im[:,xb[t]-2:xb[t]+2],axis=1)
        nz = np.flatnonzero(mid)
        yb[t] = nz[nz.size-1]
    return xt,yt,xb,yb

def calculate_radius(xc, yc, x, y):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def calculate_radius_C52(yc, x, y, centerX):
    """ calculate the distance of each data points from the center (xc, yc) """
    return np.sqrt((x-centerX)**2 + (y-yc)**2)

def fit_circle(c, x, y):
    """ calculate the algebraic distance between the 2D points and the mean circle centered at c=(xc, yc) """
    Ri = calculate_radius(*c, x, y)
    return Ri - Ri.mean()

def fit_circle_C52(c, x, y, centerX):
    """ calculate the algebraic distance between the 2D points and the mean circle centered at c=(xc, yc) """
    Ri = calculate_radius_C52(*c, x, y, centerX)
    return Ri - Ri.mean()

In [10]:
def get_circles_from_points(x, y, center, use_C52=True):
    # solve for top circle
    centerX0 = center[0]
    centerY = center[1]
    if use_C52:
        center_estimate, ier = optimize.leastsq(fit_circle_C52,centerY,args=(x,y,centerX0),col_deriv=True)
        Ri                   = calculate_radius_C52(*center_estimate,x,y,centerX)
        x_estimate = centerX0
        y_estimate = center_estimate[0]
        r_estimate = Ri.mean()
    else:
        center_estimate, ier = optimize.leastsq(fit_circle,center,args=(x,y),col_deriv=True)
        Ri                = calculate_radius(*center_top_estimate,x,y)
        x_estimate = center_estimate[0]
        y_estimate = center_estimate[1]
        r_estimate = Ri.mean()
    return x_estimate, y_estimate, r_estimate

In [11]:
# Compute a least squares estimate the top and bottom circles from lists of points along the top and bottom curves
#   for every ultrasound video (represented by a mean 2D image)

centerX = 0
for i in range(3):
    im = (im_mask[i]>0).astype(np.uint8)
    c = scipy.ndimage.measurements.center_of_mass(im)[1]
    centerX += c
    print(c)
centerX = int(np.round(centerX/3,0))

centert = (centerX, -55)
centerb = (centerX, -721)

x_top_estimate = np.zeros(num_mask)
y_top_estimate = np.zeros(num_mask)
r_top_estimate = np.zeros(num_mask)
x_bottom_estimate = np.zeros(num_mask)
y_bottom_estimate = np.zeros(num_mask)
r_bottom_estimate = np.zeros(num_mask)

for i in range(3):
    xt,yt,xb,yb = get_top_and_bottom_curve_points_C52(im_mask[i], centerX)
    
    x_top_estimate[i], y_top_estimate[i], r_top_estimate[i] = get_circles_from_points(xt, yt, centert, use_C52=True)
    x_bottom_estimate[i], y_bottom_estimate[i], r_bottom_estimate[i] = get_circles_from_points(xb, yb, centerb, use_C52=True)

    
    print(im_mask_depth[i])
    print("   Top: ", x_top_estimate[i], y_top_estimate[i], r_top_estimate[i])
    print("   Bottom: ", x_bottom_estimate[i], y_bottom_estimate[i], r_bottom_estimate[i])
    print()
    

952.6726509025195
955.0576735265394
953.6145025854096
5
   Top:  954.0 11.143492642914767 421.5909040997465
   Bottom:  954.0 -85.72432851471096 908.1117281864598

12
   Top:  954.0 27.99804506346475 227.3948431639985
   Bottom:  954.0 -336.26705460765976 1263.6184074778234

16
   Top:  954.0 -120.74286198368526 297.8172838772864
   Bottom:  954.0 162.20133192135734 889.5753071341097



In [62]:
def get_linear_map_C52(mask, xt, yt, rt, xb, yb, rb, ray_density = 0.5, blur = 0.5):
    try:
        assert(blur > 0)
    except:
        sys.exit("blur needs to be greater than zero")

    midline = xt 

    center_x = xt 
    center_y = yt

    center = np.array([center_x, center_y])

    inner_radius = rt
    outer_radius = (yb+rb)-yt

    radii = np.array([inner_radius, outer_radius])
    angle = (67.5/180)*math.pi
    
    left_angle = -angle/2
    right_angle = angle/2

    # determine the x and y sizes of the resampled image
    # from ray density. y size will be sector depth

    target_xsize = int(ray_density*(outer_radius)*angle + 0.5) # arc length (pixels) times ray density
    target_ysize = int(outer_radius - inner_radius + 0.5) # depth of US image

    # create mapping tensor

    mapping = np.zeros([target_ysize, target_xsize, 11])

    thetas = np.linspace(left_angle, left_angle+angle, target_xsize+2)
    rads = np.linspace(inner_radius, outer_radius, target_ysize+2)

    s0 = mask.shape[0]
    s1 = mask.shape[1]
    for i in range(target_xsize):
        print(i/target_xsize)
        for j in range(target_ysize): 
            
            theta = thetas[i + 1]
            rad = rads[j + 1]
            
            x = np.sin(theta)*rad + center_x
            y = np.cos(theta)*rad + center_y
            
            kernel_center_x = int(np.round(x))
            kernel_center_y = int(np.round(y))

            kernel_weights = np.zeros([3,3])

            G = multivariate_normal([x,y], np.eye(2)*blur)
            for m in range(3):
                for n in range(3):
                    i0 = kernel_center_x + m - 1
                    i1 = kernel_center_y + n - 1
                    if (i0<s0 and i1<s1 and mask[i0,i1]):
                        kernel_weights[m,n] = G.pdf([i0,i1])
                    else:
                        kernel_weights[m,n] = 0.0

            if (np.sum(kernel_weights) != 0):
                kernel_weights = kernel_weights / np.sum(kernel_weights)
            kernel_weights = kernel_weights.reshape(9)
            mapping[j,i] = np.concatenate(([kernel_center_x, kernel_center_y], kernel_weights))
        
    return mapping

In [63]:
i = 2
xt = x_top_estimate[i]
yt = y_top_estimate[i]
rt = r_top_estimate[i]
xb = x_bottom_estimate[i]
yb = y_bottom_estimate[i]
rb = r_bottom_estimate[i]
foo = get_linear_map_C52(im_mask[i], xt, yt, rt, xb, yb, rb, ray_density = 0.5, blur = 0.5)

0.0
0.001447178002894356
0.002894356005788712
0.004341534008683068
0.005788712011577424
0.00723589001447178
0.008683068017366137
0.010130246020260492
0.011577424023154847
0.013024602026049204
0.01447178002894356
0.015918958031837915
0.017366136034732273
0.01881331403762663
0.020260492040520984
0.02170767004341534
0.023154848046309694
0.024602026049204053
0.02604920405209841
0.027496382054992764
0.02894356005788712
0.030390738060781478
0.03183791606367583
0.03328509406657019
0.03473227206946455
0.0361794500723589
0.03762662807525326
0.03907380607814761
0.04052098408104197
0.041968162083936326
0.04341534008683068
0.04486251808972504
0.04630969609261939
0.04775687409551375
0.049204052098408106
0.05065123010130246
0.05209840810419682
0.053545586107091175
0.05499276410998553
0.056439942112879886
0.05788712011577424
0.059334298118668596
0.060781476121562955
0.06222865412445731
0.06367583212735166
0.06512301013024602
0.06657018813314038
0.06801736613603473
0.0694645441389291
0.070911722141823

0.6092619392185239
0.6107091172214182
0.6121562952243126
0.613603473227207
0.6150506512301013
0.6164978292329957
0.61794500723589
0.6193921852387844
0.6208393632416788
0.622286541244573
0.6237337192474675
0.6251808972503617
0.6266280752532561
0.6280752532561505
0.6295224312590448
0.6309696092619392
0.6324167872648335
0.6338639652677279
0.6353111432706223
0.6367583212735166
0.638205499276411
0.6396526772793053
0.6410998552821997
0.6425470332850941
0.6439942112879884
0.6454413892908828
0.6468885672937771
0.6483357452966715
0.6497829232995659
0.6512301013024602
0.6526772793053546
0.6541244573082489
0.6555716353111433
0.6570188133140377
0.658465991316932
0.6599131693198264
0.6613603473227206
0.662807525325615
0.6642547033285094
0.6657018813314037
0.6671490593342981
0.6685962373371924
0.6700434153400868
0.6714905933429812
0.6729377713458755
0.6743849493487699
0.6758321273516642
0.6772793053545586
0.678726483357453
0.6801736613603473
0.6816208393632417
0.683068017366136
0.6845151953690304
0.

In [70]:
img = itk.GetImageFromArray(foo.astype('f'))
itk.imwrite(img,"linear_map_depth12.mha")